In [1]:
import csv
import pandas as pd
import copy
import time
from math import sqrt

In [2]:
# Read tile IDs, group IDs, and values into data frame. Values that are undetermined are set to zero.
file = '2022_June.csv'
data = pd.DataFrame(columns=['TileID', 'GroupID', 'Value'])
with open(file, 'r') as f:
    firstLine = True
    for row in csv.reader(f, delimiter= ','):
        if firstLine:
            firstLine = False
            continue
        else:
            data.loc[len(data.index)] = list(map(int, row))
            
# create new column that contains the number of members in tileID's group            
data['GroupSize'] = data.groupby(['GroupID'])['Value'].transform('count')

data.head(10)

,TileID,GroupID,Value,GroupSize
0,0,1,0,10
1,1,2,3,6
2,2,2,0,6
3,3,2,0,6
4,4,3,0,9
5,5,3,7,9
6,6,3,0,9
7,7,3,0,9
8,8,3,0,9
9,9,3,0,9


In [3]:
def IDs2Remove(ID, val, allIDs):
    '''
    Return all tile IDs in the same group as 'ID' and/or less than 'val' taxicab distance from 'ID'.

    ARGS IN:
        - ID (int): a tile ID
        - val (int): a taxicab distance
        - allIDs (List[int]): list of all tile IDs
    ARGS OUT:
        - removeIDs (List[int]): list of IDs for which to remove 'val' as an option
     '''
    groupIDs = list(data.loc[data['GroupID'] == data.at[ID, 'GroupID']]['TileID'])
    
    numRows = int(sqrt(len(allIDs)))
    interiorIDs = []
    for tileID in allIDs:
        rowDist = abs((tileID//numRows)*numRows - (ID//numRows)*numRows) // numRows
        colDist = abs(tileID%numRows - ID%numRows)

        if rowDist + colDist < val:
            interiorIDs.append(tileID)
            
    removeIDs = groupIDs + interiorIDs
        
    return removeIDs

In [4]:
def IDsOnBorder(ID, val, allIDs):
    '''
    ARGS IN:
        - ID (int): a tile ID
        - val (int): a taxicab distance
        - allIDs (List[int]): list of all tile IDs
    ARGS OUT:
        - borderIDs (List[int]): list of IDs that are exactly 'val' taxicab distance from 'ID'
     '''
    numRows = int(sqrt(len(allIDs)))
    borderIDs = []
    for tileID in allIDs:
        rowDist = abs((tileID//numRows)*numRows - (ID//numRows)*numRows) // numRows
        colDist = abs(tileID%numRows - ID%numRows)

        if rowDist + colDist == val:
            borderIDs.append(tileID)
        
    return borderIDs

In [5]:
def removeChoice(ID, val, allIDs, options):
    '''
    Remove 'val' from the list of options of tile IDs relevant to 'ID' (group and/or less than 'val' 
    taxicab distance away) and set ID's value to 'val'.

    ARGS IN:
        - ID (int): a tile ID
        - val (int): a possible value for tile 'ID'
        - allIDs (List[int]): list of all tile IDs
    ARGS OUT:
        - options (List[List[int]]): list of possible values for each ID
    '''
    removeIDs = IDs2Remove(ID, val, allIDs)
    for tileID in removeIDs:
        try:
            options[tileID].remove(val)
        except (ValueError, AttributeError) as e:
            pass
        
    options[ID] = val
    
    return options

In [6]:

def generateInitialOptions(data):
    '''
    Refine the dataframe column 'Options', where each entry is a list containing all possible values for
    the corresponding tile ID.

    ARGS IN:
        - data (pd.DataFrame): has columns 'TileID', 'GroupID', 'Value', 'GroupSize', 'Options'
    ARGS OUT:
        - data (pd.DataFrame): modified input argument
    '''
    numSquares = len(data.index)
    options = [0]*numSquares
    for tileID,val in enumerate(data['Value']):
        if val == 0:
            options[tileID] = list(range(1, data.at[tileID, 'GroupSize'] + 1))
        else:
            options[tileID] = val
    
    for tileID,val in enumerate(data['Value']):
        if val != 0:
            options = removeChoice(tileID, val, list(data['TileID']), options)
            
    data['Options'] = options
    
    return data

In [7]:
data = generateInitialOptions(data)
data.head(10)

,TileID,GroupID,Value,GroupSize,Options
0,0,1,0,10,"[1, 2, 4, 5, 8, 9, 10]"
1,1,2,3,6,3
2,2,2,0,6,"[1, 2, 5, 6]"
3,3,2,0,6,"[1, 2, 5, 6]"
4,4,3,0,9,"[1, 2, 3, 5, 6, 8, 9]"
5,5,3,7,9,7
6,6,3,0,9,"[1, 2, 3, 4, 5, 6, 8, 9]"
7,7,3,0,9,"[1, 2, 3, 4, 5, 6, 8, 9]"
8,8,3,0,9,"[1, 2, 3, 4, 5, 6, 8, 9]"
9,9,3,0,9,"[1, 2, 3, 4, 5, 8, 9]"


In [8]:
def makeAMove(options):
    '''
    "Make a move" by guessing a value for a particular tile ID. Returns an updated list of options lists as well as 
    a boolean (which we'll call "conflict") that indicates whether we've created an impossible board.

    ARGS IN:
        - options (List[List[int]]): list of possible values for each tile ID
    ARGS OUT:
        - options (List[List[int]]): modified input argument
        - conflict (bool): indicates whether we've created a board that doesn't work
    '''

    conflict = False
    
    ###################################################### 
    # First see if we need to return before making a move.
    ######################################################
    
    if not all(options):  # return if there are spaces with no available options; there is a conflict
        conflict = True
        return options, conflict
    elif all([type(x) == int for x in options]):  # return if the puzzle is solved; there's no conflict
        return options, conflict
    
    # check the "taxicab border distance" around already-determined squares to see if there are conflicts
    numSquares = len(options)
    for ID,val in enumerate(options):
        if type(val) == int:  # value has been determined
            borderIDs = IDsOnBorder(ID, val, list(range(numSquares)))
            
            # if all border IDs have been determined and none of them is 'val', there is a conflict
            if all([type(options[i]) == int for i in borderIDs]) and all([options[i] != val for i in borderIDs]):
                conflict = True
                return options, conflict
    
    ###############################
    # Now we can try to make moves.
    ###############################
    
    unmoved = dict()  # store undecided tile IDs and their values
    for ID,opts in enumerate(options):
        if type(opts) == list:
            unmoved[ID] = opts

    # pick out the first ID with the fewest number of options
    numOptions = 1
    guessIDs = []
    while not guessIDs:
        guessIDs = [ID for ID in unmoved if len(unmoved[ID]) == numOptions]
        numOptions += 1
        
    guessID = guessIDs[0]
    
    # try each of guessID's possible values until one works
    for val in unmoved[guessID]:
        newOptions = copy.deepcopy(options)
        newOptions, conflict = makeAMove(removeChoice(guessID, val, list(range(numSquares)), newOptions))
            
        if conflict:  # occurs if the new move created an impossible board
            continue
        else: 
            return newOptions, conflict
    
    # a catch-all for if trying all of guessID's possible values fails
    conflict = True
    return options, conflict

In [9]:
def puzzleSolver(data):
    '''
    ARGS IN:
        - data (pd.DataFrame): has columns 'TileID', 'GroupID', 'Value'
    ARGS OUT:
        - data (pd.DataFrame): modified input argument with new (solved) columns 'Value' and 'Options'
    '''

    data = generateInitialOptions(data)
    
    soln, conflict = makeAMove(list(data['Options']))
    
    data['Value'] = soln
    data['Options'] = soln

    return data

In [10]:
t0 = time.perf_counter()
data = puzzleSolver(data)
t1 = time.perf_counter()

numRows = int(sqrt(len(data.index)))

solnString = ''
finalAnswer = 0
tempMult = 1
for tileID,val in enumerate(data['Value']):
    solnString += str(val) + ' '
    tempMult *= val
    
    if tileID % numRows == numRows - 1:
        solnString += '\n'
        finalAnswer += tempMult
        tempMult = 1

print('Code takes ' + str(t1-t0) + ' seconds to run.\n')
print('Solution board:\n')
print(solnString)
print('Final answer to puzzle:\n')
print(finalAnswer)

Code takes 67.6722338 seconds to run.

Solution board:

4 3 6 5 3 7 4 9 6 5 
8 10 2 4 1 1 2 3 8 2 
9 2 3 2 1 2 5 1 2 4 
5 7 2 1 2 6 3 1 1 3 
6 3 1 1 3 2 1 4 2 7 
1 1 4 5 1 1 1 3 5 6 
3 1 2 3 2 4 2 1 2 3 
4 2 1 1 1 1 3 1 4 9 
5 8 3 4 2 1 6 2 3 8 
7 6 9 10 5 3 4 7 2 5 

Final answer to puzzle:

24405360
